In [1]:
import pandas as pd

In [33]:
city_df = pd.read_csv("AirQuality/Dataset/GroundTruth_PM2_5_2023.csv")
temp_df = pd.read_csv("AirQuality/RQ3/Dataset/Temperature.csv")

print(city_df.columns)
print(temp_df.columns)

Index(['city', 'state', 'YearMonth', 'AT', 'BP', 'PM2.5', 'RF', 'VWS', 'WD',
       'WS', 'latitude', 'longitude', 'latitude_osm', 'longitude_osm'],
      dtype='object')
Index(['date', 'station', 'city', 'latitude', 'longitude', 'state',
       'Min_Temp_C', 'Max_Temp_C', 'Avg_Temp_C'],
      dtype='object')


In [35]:
temp_df.head()

,date,station,city,latitude,longitude,state,Min_Temp_C,Max_Temp_C,Avg_Temp_C
0,2021-01-01,"32Bungalows, Bhilai - CECB",Bhilai,21.194815,81.314770,Chhattisgarh,NaN,NaN,NaN
1,2021-01-01,"AIIMS, Raipur - CECB",Raipur,21.258815,81.578979,Chhattisgarh,NaN,NaN,NaN
2,2021-01-01,"Adarsh Nagar, Jaipur - RSPCB",Jaipur,26.902909,75.836858,Rajasthan,NaN,NaN,NaN
3,2021-01-01,"Airoli, Navi Mumbai - MPCB",Navi Mumbai,19.149400,72.998600,Maharashtra,NaN,NaN,NaN
4,2021-01-01,"Alandi, Pune - IITM",Pune,18.675076,73.892743,Maharashtra,NaN,NaN,NaN


In [36]:
temp_df['date'] = pd.to_datetime(temp_df['date'],errors='coerce')
temp_df = temp_df[temp_df['date'].dt.year == 2023]

In [37]:
agg_temp_df = temp_df.groupby(['city','state','date']).agg({
    'Min_Temp_C':'mean',
    'Max_Temp_C': 'mean',
    'Avg_Temp_C': 'mean'
}).reset_index()

valid_pairs = set(zip(city_df['city'],city_df['state']))
filtered_temp_df = agg_temp_df[agg_temp_df.apply(lambda row: (row['city'],row['state']) in valid_pairs,axis=1)]

print(filtered_temp_df.head())
print(f"Rows retained: {len(filtered_temp_df)}")
print(f"Unique city,state pairs: {filtered_temp_df[['city', 'state']].drop_duplicates().shape[0]}")

     city          state       date  Min_Temp_C  Max_Temp_C  Avg_Temp_C
365  Agra  Uttar Pradesh 2023-01-01    6.966667   19.333333   12.439583
366  Agra  Uttar Pradesh 2023-01-02    6.983333   17.883333   11.646528
367  Agra  Uttar Pradesh 2023-01-03    6.683333   15.833333   10.447917
368  Agra  Uttar Pradesh 2023-01-04    5.850000   13.866667    9.687500
369  Agra  Uttar Pradesh 2023-01-05    6.400000   12.966667    9.440972
Rows retained: 52560
Unique city,state pairs: 144


In [38]:
filtered_temp_df = filtered_temp_df.copy()
filtered_temp_df['date'] = pd.to_datetime(filtered_temp_df['date'])

filtered_temp_df['year'] = filtered_temp_df['date'].dt.year
filtered_temp_df['month'] = filtered_temp_df['date'].dt.month

monthly_temp_df = (
    filtered_temp_df.groupby(['city','state','year','month'],as_index=False).agg({
        'Min_Temp_C':'mean',
        'Max_Temp_C' : 'mean',
        'Avg_Temp_C': 'mean'
    })
)

print(f"Total rows: {len(monthly_temp_df)}")

Total rows: 1728


In [39]:
city_df['YearMonth'] = pd.to_datetime(city_df['YearMonth']).copy()
city_df['year'] = city_df['YearMonth'].dt.year
city_df['month'] = city_df['YearMonth'].dt.month

merged_df = city_df.merge(
    monthly_temp_df,
    on=['city', 'state', 'year', 'month'],
    how='left'
)

merged_df.drop(['year', 'month'], axis=1, inplace=True)

In [40]:
merged_df.head()

,city,state,YearMonth,AT,BP,PM2.5,RF,VWS,WD,WS,latitude,longitude,latitude_osm,longitude_osm,Min_Temp_C,Max_Temp_C,Avg_Temp_C
0,Agra,Uttar Pradesh,2023-01-01,12.409831,746.468453,44.641356,0.006923,-0.146221,148.984383,1.007488,27.184136,77.990717,27.175255,78.009816,7.819355,18.948387,12.976658
1,Agra,Uttar Pradesh,2023-02-01,20.287904,744.916665,19.689583,0.003722,-0.139412,162.291339,1.057285,27.184136,77.990717,27.175255,78.009816,12.958929,28.667262,20.043180
2,Agra,Uttar Pradesh,2023-03-01,24.130233,743.951707,16.330529,0.006760,-0.142067,140.689598,0.991312,27.184136,77.990717,27.175255,78.009816,17.995161,30.781183,23.842854
3,Agra,Uttar Pradesh,2023-04-01,29.194105,741.947767,17.066000,0.004371,-0.187498,151.361473,1.047574,27.184136,77.990717,27.175255,78.009816,21.727222,36.213333,28.564352
4,Agra,Uttar Pradesh,2023-05-01,30.769024,740.259316,23.099587,0.012180,-0.173758,155.387008,1.180217,27.184136,77.990717,27.175255,78.009816,24.633333,38.102151,31.042384


In [41]:
# Compute difference
merged_df['AT_AvgTemp_Diff'] = merged_df['AT'] - merged_df['Avg_Temp_C']

# Compute absolute difference (for magnitude comparison)
merged_df['Abs_AT_AvgTemp_Diff'] = merged_df['AT_AvgTemp_Diff'].abs()

# Find the row with the highest absolute difference
max_diff_row = merged_df.loc[merged_df['Abs_AT_AvgTemp_Diff'].idxmax()]


In [42]:
print("Max absolute difference:", max_diff_row['Abs_AT_AvgTemp_Diff'])
print("Corresponding row:")
print(max_diff_row)


Max absolute difference: 11.824827152520545
Corresponding row:
city                              Bilaspur
state                         Chhattisgarh
YearMonth              2023-01-01 00:00:00
AT                               33.214074
BP                              746.176942
PM2.5                            25.470468
RF                                     0.0
VWS                                    NaN
WD                              178.403744
WS                                0.136392
latitude                          22.08815
longitude                         82.13737
latitude_osm                     22.163849
longitude_osm                    82.136592
Min_Temp_C                        15.83871
Max_Temp_C                       27.841935
Avg_Temp_C                       21.389247
AT_AvgTemp_Diff                  11.824827
Abs_AT_AvgTemp_Diff              11.824827
Name: 276, dtype: object


In [43]:
merged_df.drop(['AT_AvgTemp_Diff', 'Abs_AT_AvgTemp_Diff'], axis=1, inplace=True)
merged_df.to_csv("merged_city_temp.csv", index=False)